In [91]:
from google.colab import drive
drive.mount('/content/drive')
! wget -c -P '/content/' https://banks.data.fdic.gov/docs/All%20Financial%20Reports.xlsx

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
--2025-03-14 17:13:26--  https://banks.data.fdic.gov/docs/All%20Financial%20Reports.xlsx
Resolving banks.data.fdic.gov (banks.data.fdic.gov)... 160.1.183.41, 3.30.133.56, 2600:1f12:18a:7d00:2834:1f9c:e20:50c1, ...
Connecting to banks.data.fdic.gov (banks.data.fdic.gov)|160.1.183.41|:443... connected.
HTTP request sent, awaiting response... 500 Internal Server Error
2025-03-14 17:13:26 ERROR 500: Internal Server Error.



In [154]:
'''
Beck Buchanan - Senior Thesis
Python Script for Retrieval of FDIC Data
'''
# MODULES AND LIBRARIES --------------------------------------------------------
import requests
import time
import csv
import pandas as pd
from IPython import display
import textwrap
from rich.console import Console
import sys

# FORMATTING -------------------------------------------------------------------
console = Console()
def stop():
  '''Takes no input params, but prompts user for an input, and clears display
  once user submits their input
  '''
  speedy_print(f"\n{'Press enter to continue...':^135}\n\n")
  input()
  display.clear_output()

def speedy_print(string):
  '''Takes input from a string and prints each character onto
  a single line, separated by a short pause.
  '''
  for char in string:
      sys.stdout.write(char)
      sys.stdout.flush()
      time.sleep(0.005)

# FUNCTION DEFINITIONS ---------------------------------------------------------
def list_vars(f_vars):
  count = 4
  inc = count
  user = ""
  while user.lower() != 'done':
    dict_vars = f_vars[count - inc:count].to_dict(orient='index')
    for var in dict_vars:

      for val in dict_vars[var]:
        print(f"\t{val}: {dict_vars[var][val]}")
      print('=======')
    time.sleep(0.01)
    user = input()
    display.clear_output()
    count += inc

def search(df):
  print("Please input selection. If not filtering on characteristic, \n\
  please enter a blank string")
  title = input('Input Title Keywords: \n')
  keyword = input('Input Definition Keywords: \n')
  unit = input('Input Desired Unit ($/%): \n')

  while unit not in ['$', '%', ""]:
    unit = input("Please Enter '%' or '$': \n")
  time = input('Input Desired Frequency (Qtly/YTD): \n')
  while time not in ['Qtly', 'YTD', ""]:
    time = input("Please Enter 'Qtly' or 'YTD': \n")

  if title != "":
    df = df[df['Title'].str.contains(keyword)]
  if keyword != "":
    df = df[df['Definition'].str.contains(keyword)]
  if unit != "":
    df = df[df['Units'].str.contains(unit)]
  if time != "":
    df = df[df['Frequency'].str.contains(time)]

  filters = {"Title": title, "Keyword":keyword, "Unit":unit, "Frequency":time}
  display.clear_output()

  print("Retrieving variables, filtered by: ")
  count = 0
  for filter in filters:
    if filters[filter] != "":
      print(f"{filter}:\t {filters[filter]}")
    else:
      count += 1
  if count == 4:
    print("\n\tNo Filters")
  print("\n===========\n")

  list_vars(df)

def user_list():
  print("\nPlease enter desired field values. \n\
  The program will return the requested data plus REPDATE and Firm Identifiers\n\
   for your requested years into a file named '[YOUR_FIELD]data.csv'")

  entries = []
  user_entry = input('Enter a field to request ')
  entries.append(user_entry.upper())
  while user_entry != 'done':
    print("Enter another field to request. If you have no more fields, enter 'done' ")
    user_entry = input()
    if user_entry != 'done':
      entries.append(user_entry.upper())
  display.clear_output()
  return entries

def wrap_text(text, width):
    return '\n'.join(textwrap.wrap(text, width=width))

def menu(df):
  responses = {"DATA": "Go to Data Gathering Tool",\
               "LIST": "See full list of variables",\
               "SEARCH": "Use Filters to search for variables",\
               "EXIT": "Exit the program"}

  console.print(f"\n\n{'[bold]FDIC API Data Retrieval Tool[/bold]':^150}")
  #console.print(f"{'[italic]Press Enter to Begin[/italic]':^153}\n\n")
  stop()
  exit = True
  while exit:
    display.clear_output()
    console.print(f"\n\n{'[bold]Main Menu[/bold]':^150}\n")
    console.print(f"\n\n{'[italic]What would you like to do?[/italic]':^155}\n")
    console.print(f"\n\n{'[italic]For a list of choices, type OPTIONS [/italic]':^155}\n\n")
    time.sleep(0.1)
    i1 = input()
    while i1.upper() not in ["DATA", "SEARCH", "LIST", "EXIT"]:
      if i1.upper() == "OPTIONS":
        for key in responses:
          print(f"{key:>60}: {responses[key]}")
        print()
      i1 = input()

    if i1.upper == "EXIT":
      exit = False

    if i1.upper() == "SEARCH":
      search(df)

    if i1.upper() == "LIST":
      list_vars(df)

    if i1.upper() == "DATA":
      f = input("Yearly or Quarterly? (Y/Q):\n ")
      f = f.upper()
      while f not in ["Y", "Q"]:
        f = input("Yearly or Quarterly? (Y/Q):\n ")
        f = f.upper()
      gather_data(f)

def yearly_data(entries, bclass='N', file_path='/content/drive/MyDrive/Thesis_Raw_Data/',\
                start_year=1984,end_year=2022):
  store_year = end_year
  path_tag = "data.csv"
  for entry in entries:
      # Go through each year in the range
      field = {
          'fields': f'{entry},REPDTE,CERT',
          'sort_by': 'BKCLASS',
          'sort_order': 'ASC',
          'limit': '10000',
          'format': 'csv',
          'download': 'false',
          'filename': 'bank_data'
      }
      all_data = []  # List to store all data before writing
      while end_year >= start_year:
          # Format date properly as YYYYMMDD
          date_str = f"{end_year}1231"
          url = f"https://banks.data.fdic.gov/api/financials?filters=BKCLASS:{bclass} AND REPDTE:{date_str}"
          try:
              print(f"Requesting data for {date_str}...")
              r = requests.get(url, params=field)
              r.raise_for_status()

              # Check if we got any data
              data = r.text.splitlines()
              if len(data) > 1:  # More than just header
                  all_data.extend(data if end_year == store_year else data[1:])
                  print(f"Successfully retrieved {len(data)-1} records for {end_year}")
                  display.clear_output()

              else:
                  print(f"No data found for {end_year}")

          except requests.exceptions.RequestException as e:
              print(f"Error retrieving data for {end_year}: {e}")

          end_year -= 1

      # Write all collected data at once
      if all_data:
          entry_path = file_path + f'{entry}' + path_tag
          with open(entry_path, 'w', newline='') as f:
              f.write('\n'.join(all_data))
          print(f"Wrote {len(all_data)} lines to {entry_path}")

      # Reset year for next field
      end_year = store_year

def quarterly_data(entries, bclass='N', file_path='/content/drive/MyDrive/Thesis_Raw_Data/',\
                start_year=1984,end_year=2022):

  qtrs = ['0331', '0630', '0930', '1231']
  store_year = end_year
  path_tag = "data.csv"

  for qtr in qtr:
    for entry in entries:
      # Go through each year in the range
      field = {
          'fields': f'{entry},REPDTE,CERT',
          'sort_by': 'BKCLASS',
          'sort_order': 'ASC',
          'limit': '10000',
          'format': 'csv',
          'download': 'false',
          'filename': 'bank_data'
      }
      all_data = []  # List to store all data before writing
      while end_year >= start_year:
          # Format date properly as YYYYMMDD
          date_str = f"{end_year}{qtr}"
          url = f"https://banks.data.fdic.gov/api/financials?filters=BKCLASS:{bclass} AND REPDTE:{date_str}"
          try:
              print(f"Requesting data for {date_str}...")
              r = requests.get(url, params=field)
              r.raise_for_status()

              # Check if we got any data
              data = r.text.splitlines()
              if len(data) > 1:  # More than just header
                  all_data.extend(data if end_year == store_year else data[1:])
                  print(f"Successfully retrieved {len(data)-1} records for {end_year}")
                  display.clear_output()

              else:
                  print(f"No data found for {end_year}")

          except requests.exceptions.RequestException as e:
              print(f"Error retrieving data for {end_year}: {e}")

          end_year -= 1

      # Write all collected data at once
      if all_data:
          entry_path = file_path + f'{entry}' + path_tag
          with open(entry_path, 'w', newline='') as f:
              f.write('\n'.join(all_data))
          print(f"Wrote {len(all_data)} lines to {entry_path}")

      # Reset year for next field
      end_year = store_year

  pass

def gather_data(frequency, bclass='N', file_path='/content/drive/MyDrive/Thesis_Raw_Data/',\
                start_year=1984,end_year=2022):
  store_year = end_year
  path_tag = 'data.csv'
  entries = user_list()
  if frequency[0].upper() == 'Y':
    yearly_data(entries)
  else:
    quarterly_data(entries)

if True:
  # IMPORTING AND PARSING VARIABLE DEFINITION SHEET ------------------------------
  f_vars = pd.read_excel('/content/All Financial Reports.xlsx', 'Reference-Variables&Definitions')
  f_vars = f_vars.astype(str)
  f_vars['Definition'] = f_vars['Definition'].apply(lambda x: '( ---, - )' + x if not str(x).startswith('(') else x)
  f_vars[['Frequency','Units', 'Definition']] = f_vars['Definition'].str.split(r"\)|,", expand=True, n=2)
  f_vars['Frequency'] = f_vars['Frequency'].str.lstrip('( ')
  f_vars['Definition'] = f_vars['Definition'].str.lstrip()
  f_vars['Definition'] = f_vars['Definition'].apply(lambda x: wrap_text(x, 80))

# MAIN CALL --------------------------------------------------------------------
menu(f_vars)

Main Menu

What would you like to do?

For a list of choices, type OPTIONS 

KeyboardInterrupt: Interrupted by user